In [1]:
import os
import time
os.getcwd()

'/Users/yikaitang/Documents/GitHub/puppet_training/examples/puppet_training/notebooks'

In [3]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
    sys.path.insert(0, module_path)
# endregion       #
# --------------- #

from examples.puppet_training.utils import (
    init_log, parse_args, load_config,
    create_models,
    create_agents,
    create_entities,
)
from examples.puppet_training.env import puppet_training
from agentarium.logging_utils import GameLogger
from agentarium.models import human_player
from agentarium.utils import visual_field_sprite, image_from_array
from IPython.display import clear_output
from matplotlib import pyplot as plt
import random
import argparse
import torch
import numpy as np


config_path = '../configs/records/puppet_training_full_partner_selection_env_fixed_config_backup.yaml'

cfg = load_config(argparse.Namespace(config=config_path))

# set seed 
cfg.seed = 12
random.seed(cfg.seed)
np.random.seed(cfg.seed)
torch.manual_seed(cfg.seed)
torch.cuda.manual_seed_all(cfg.seed)
cfg.exp_name = cfg.exp_name + f'_seed{cfg.seed}'


models_agent = create_models(cfg)
agents = create_agents(cfg, models_agent)

# change agent attribute
for agent in agents:
    if agent.ixs == 0:
        agent.role = 'decider'
    else:
        agent.role = 'partner'
    agent.value_dict = {'Gem': 2, 'Bone': 2, 'Coin': 2, 'Agent': 0, 'Wall':-1, 'EmptyObject':0}

# assign roles
if not cfg.train_partners:
    print('not train partners')
    for agent in agents:
        if agent.ixs == 0:
            agent.role = 'decider'
            agent.can_see_others_worldview = True
        else:
            agent.role = 'partner'
            agent.can_see_others_worldview = False
        if agent.ixs == 1:
            agent.resource_val = {'median': {'Gem': 2, 'Bone': 2, 'Coin': 2, 'Agent': 0, 'Wall':-1, 'EmptyObject':0},
            'var': {'Gem': 3, 'Bone': 3, 'Coin': 3, 'Agent': 0, 'Wall':0, 'EmptyObject':0}}
        elif agent.ixs == 2:
            agent.resource_val = {'median': {'Gem': 2, 'Bone': 2, 'Coin': 2, 'Agent': 0, 'Wall':-1, 'EmptyObject':0},
            'var': {'Gem': 0, 'Bone': 0, 'Coin': 0, 'Agent': 0, 'Wall':0, 'EmptyObject':0}}
else:
    for agent in agents:
        agent.role = 'decider'
        agent.can_see_others_worldview = False
        agent.resource_val = {'median': {'Gem': 2, 'Bone': 2, 'Coin': 2, 'Agent': 0, 'Wall':-1, 'EmptyObject':0},
        'var': {'Gem': 0, 'Bone': 0, 'Coin': 0, 'Agent': 0, 'Wall':0, 'EmptyObject':0}}

entities = create_entities(cfg)
env = puppet_training(cfg, agents, entities, is_partner_selection_env=True)
env.full_partner_selection = True

for ixs, agent in enumerate(agents):
    # agent.model = human_player.ModelHumanPlayer(
    #      action_space=4, 
    #      state_size = cfg.model.iqn.parameters.state_size, 
    #      extra_percept_size = cfg.model.iqn.parameters.extra_percept_size,
    #      memory_size=1, 
    #      name=f'human {ixs}')

    # agent.model = models_agent[0]
    
    agent.model.load(
        #  f'../models/checkpoints/puppet_training_only_partners_env_agent{0}_iRainbowModel.pkl')
        f'../models/checkpoints/puppet_training_full_partner_selection_env_fixed_agent{agent.ixs}_iRainbowModel.pkl')
    # agent.model.load(
    #      '../models/checkpoints/fixed_punishment_rate_1.0_oneAs_size15_init_spawn_0.2_agent0_iRainbowModel.pkl'
    # )
    # agent.model.epsilon = 0.01


for epoch in range(1):
    # Reset the environment at the start of each epoch
        env.reset()
        images_anim = []
        # for agent in env.agents:
        #     agent.reset(env)
        random.shuffle(agents)

        scores = GameLogger(max_epochs=1)

        done = 0 
        turn = 0
        losses = 0
        game_points = [0 for _ in range(len(agents))]

        while not done:

            # env.check_and_close_gate()

            turn = turn + 1

            entities = env.get_entities_for_transition()
            # Entity transition
            for entity in entities:
                entity.transition(env)

            img = visual_field_sprite(env.world, tile_size = env.tile_size)
            clear_output(wait = True)
            # scores.pretty_print(
            #      'jupyter-mode',
            #      epoch = epoch,
            #      turn = turn,
            #      reward = game_points,
            # )
            
            img = image_from_array(img)
            time.sleep(0.1)
            plt.imshow(img)
            plt.show()
            images_anim.append(img)
            print(game_points)


            # Agent transition
            for agent in agents:

                (state,
                action,
                reward,
                next_state,
                done_
                ) = agent.transition(env)

                img = visual_field_sprite(env.world, tile_size = env.tile_size)
                img = image_from_array(img)
                # print(agent.model.name)
                # plt.imshow(img)
                plt.show()
                # if agent.ixs == 0:
                #      images_anim.append(img)

                if turn >= cfg.experiment.max_turns or done_:
                    done = 1

                game_points[agent.ixs] += reward

            env.check_and_close_gate()

print(agent.encounters)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
not train partners


ValueError: could not broadcast input array from shape (224,) into shape (1568,)

In [ ]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
    sys.path.insert(0, module_path)
# endregion       #
# --------------- #

from examples.puppet_training.utils import (
    init_log, parse_args, load_config,
    create_models,
    create_agents,
    create_entities,
)
config_path = '../configs/config_fixed_rate_no_vote.yaml'
from examples.puppet_training import agents, entities
from examples.puppet_training.env import puppet_training
from examples.puppet_training.utils import inspect_the_env
from agentarium.logging_utils import GameLogger
from agentarium.models import human_player
from agentarium.utils import visual_field_sprite, image_from_array
from IPython.display import clear_output
from matplotlib import pyplot as plt
import random
import argparse

cfg = load_config(argparse.Namespace(config=config_path))

models = create_models(cfg)
agents = create_agents(cfg, models)
entities = create_entities(cfg)
env = puppet_training(cfg, agents, entities)

# instantiate the performance logger
performance = []

# change value of the entity
reward_set = [1, 0, 0]
for entity in env.entities:
    if entity.name == 'Gem':
        entity.value = reward_set[0]
    elif entity.name == 'Bone':
        entity.value = reward_set[1]
    elif entity.name == 'Coin':
        entity.value = reward_set[2]



for ixs, agent in enumerate(agents):
    agent.model = human_player.ModelHumanPlayer(
         action_space=8, 
         state_size = cfg.model.iqn.parameters.state_size, 
         extra_percept_size = cfg.model.iqn.parameters.extra_percept_size,
         memory_size=1, 
         name=f'human {ixs}')
    # agent.model.load(
    #      f'../models/checkpoints/fixed_punishment_rate_0.75_twoAs_extra_percept_v2_higher_harm_gem_has_value_save_model_agent{ixs}_iRainbowModel_20241127-04111732699956.pkl')
    # agent.model.load(
    #      '../models/checkpoints/fixed_punishment_rate_1.0_oneAs_size15_init_spawn_0.2_agent0_iRainbowModel.pkl'
    # )
    # agent.model.epsilon = 0.01


for epoch in range(1):
    # Reset the environment at the start of each epoch
    env.reset()
    images_anim = []
    # for agent in env.agents:
    #     agent.reset(env)
    random.shuffle(agents)

    scores = GameLogger(max_epochs=1)
    
    env.cache['harm'] = [0 for _ in range(len(agents))]

    done = 0 
    turn = 0
    losses = 0
    game_points = [0 for _ in range(len(agents))]

    while not done:

        turn = turn + 1

        entities = env.get_entities_for_transition()
        # Entity transition
        for entity in entities:
            entity.transition(env)

        img = visual_field_sprite(env.world, tile_size = env.tile_size)
        clear_output(wait = True)
        # scores.pretty_print(
        #      'jupyter-mode',
        #      epoch = epoch,
        #      turn = turn,
        #      reward = game_points,
        # )
        
        img = image_from_array(img)
        time.sleep(0.1)
        plt.imshow(img)
        plt.show()
        images_anim.append(img)
        print(game_points)


        # Agent transition
        for agent in agents:

            (state,
            action,
            reward,
            next_state,
            done_
            ) = agent.transition(env)

            img = visual_field_sprite(env.world, tile_size = env.tile_size)
            img = image_from_array(img)
            # print(agent.model.name)
            # plt.imshow(img)
            plt.show()
            # if agent.ixs == 0:
            #      images_anim.append(img)

            if turn >= cfg.experiment.max_turns or done_:
                done = 1

            exp = (1, (state, action, reward, next_state, done))
            # agent.episode_memory.append(exp)

            game_points[agent.ixs] += reward

            print(agent.location)

        img = visual_field_sprite(env.world, tile_size = env.tile_size)
        clear_output(wait = True)
        img = image_from_array(img)
        time.sleep(0.1)
        plt.imshow(img)
        plt.show()
    
    # record the performance of the agents
    total_encounters = {entity:0 for entity in vars(cfg.entity)}
    for agent in agents:
        for entity in vars(cfg.entity):
            total_encounters[entity] += agent.encounters[entity]

    performance[reward_set] = total_encounters

    # print the performance of the agents
    for reward_set, encounters in performance.items():
        print('==========================')
        print(f"Reward set: {reward_set}")
        for entity, count in encounters.items():
            print(f"{entity}: {count}")
        print("\n")





In [79]:
from PIL import Image
import imageio.v3 as iio
def create_gif_from_arrays(image_arrays, output_path, duration=100, loop=0):
    """
    Create a GIF from a sequence of images in NumPy array format.

    Args:
        image_arrays (list of np.ndarray): Sequence of images as NumPy arrays.
        output_path (str): Path to save the output GIF.
        duration (int): Duration of each frame in milliseconds (controls speed).
        loop (int): Number of times the GIF should loop (0 for infinite).
    """
    # Convert NumPy arrays to PIL Images
    if type(image_arrays[0]) != Image.Image:
        pil_images = [Image.fromarray(img) for img in image_arrays]
    else:
        pil_images = image_arrays
    
    # Save as GIF
    pil_images[0].save(
        output_path,
        save_all=True,
        append_images=pil_images[1:],
        duration=duration,
        loop=loop
    )
    print(f"GIF saved at {output_path}")


def create_gif_iio(images, output_path, fps=10):
    """
    Create a GIF from a list of image arrays using `imageio`.

    Args:
        images (list of numpy.ndarray): A list of image arrays (e.g., RGB or grayscale).
        output_path (str): Path to save the generated GIF.
        fps (int): Frames per second, controlling the speed of the GIF.
    """
    # Calculate the duration per frame in seconds
    duration_per_frame = 1 / fps
    
    # Save the GIF
    iio.imwrite(output_path, images, format="GIF", duration=duration_per_frame)
    print(f"GIF saved at {output_path}")


from PIL import Image, ImageDraw

from PIL import Image

def create_gif_from_pil_force_clean(image_list, output_path, duration=100, loop=0, background_color=(0, 0, 0)):
    """
    Create a GIF from a list of `PIL.Image.Image` objects, ensuring no shadow artifacts.
    Ensures each frame has a fresh background.

    Args:
        image_list (list of PIL.Image.Image): Sequence of PIL images.
        output_path (str): Path to save the output GIF.
        duration (int): Duration of each frame in milliseconds (controls speed).
        loop (int): Number of times the GIF should loop (0 for infinite).
        background_color (tuple): RGB color to reset the background for each frame.
    """
    # Determine the size of the images
    width, height = image_list[0].size

    # Ensure all frames are re-rendered on a clean background
    cleaned_frames = []
    for img in image_list:
        # Create a blank canvas for each frame
        blank_frame = Image.new(mode=img.mode, size=(width, height), color=background_color)
        
        # Convert the image to RGBA to handle transparency (if needed)
        if img.mode != "RGBA":
            img = img.convert("RGBA")
        
        # Paste the current frame onto the blank canvas
        blank_frame.paste(img, (0, 0), mask=img if img.mode == "RGBA" else None)
        cleaned_frames.append(blank_frame)

    # Save as GIF
    cleaned_frames[0].save(
        output_path,
        save_all=True,
        append_images=cleaned_frames[1:],
        duration=duration,
        loop=loop
    )
    print(f"GIF saved at {output_path}")


In [ ]:
# from examples.puppet_training.utils import create_gif_from_arrays
print(type(images_anim[0]) == Image.Image)
create_gif_from_pil_force_clean(images_anim, 'movie_punishment_0.75_gem_has_value_diff_models.gif', 200)


True
GIF saved at movie_punishment_0.75_gem_has_value_diff_models.gif
